In [27]:
import pandas as pd
import numpy as np

from  sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [2]:
all_data_df = pd.read_csv("https://raw.githubusercontent.com/albemlee/ai4all_nhanes/main/notebooks/all_data_df.csv")

/Volumes/Active/GitHub/ai4all_nhanes/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (600,601) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29902 entries, 0 to 29901
Columns: 674 entries, SEQN to WHQ150
dtypes: float64(672), object(2)
memory usage: 153.8+ MB


In [4]:
features = [
    #diabetes
    'DID040','DIQ160','DIQ170','DIQ172','DIQ175A','DIQ175B','DIQ175C',
    'DIQ175D','DIQ175E','DIQ175F','DIQ175G','DIQ175H','DIQ175I','DIQ175J','DIQ175K',
    'DIQ175L', 'DIQ175M','DIQ175N','DIQ175O','DIQ175P','DIQ175Q','DIQ175R','DIQ175S',
    'DIQ175T','DIQ175U','DIQ175V','DIQ175W','DIQ180','DIQ050','DID060','DIQ060U',
    'DIQ070','DIQ230','DIQ240','DID250','DID260','DIQ260U','DIQ275','DIQ280','DIQ291',
    'DIQ300S','DIQ300D','DID310S','DID310D','DID320','DID330','DID341','DID350',
    'DIQ350U','DIQ360','DIQ080', 
    #sleep disorder
    'SEQN','SLQ050', 
    #physical activity
    'PAQ605','PAQ610','PAD615','PAQ620','PAQ625','PAD630','PAQ635','PAQ640','PAD645',
    'PAQ650','PAQ655','PAD660','PAQ665','PAQ670','PAD675','PAD680','PAQ706','PAQ710',
    'PAQ715', 
    #weight history
    'WHD010','WHD020','WHQ030','WHQ040','WHD050','WHQ060','WHQ070','WHD080A','WHD080B',
    'WHD080C','WHD080D','WHD080E','WHD080F','WHD080G','WHD080H','WHD080I','WHD080J',
    'WHD080K','WHD080M','WHD080N','WHD080O','WHD080P','WHD080Q','WHD080R','WHD080S',
    'WHD080T','WHD080L','WHD110','WHD120','WHD130','WHD140','WHQ150', 
    #early childhood
    'ECD010','ECQ020','ECD070A','ECD070B','ECQ080','ECQ090','WHQ030E','MCQ080E',
    'ECQ150', 
    #alcohol issues
    'ALQ101','ALQ110','ALQ120Q','ALQ120U','ALQ130','ALQ141Q','ALQ141U','ALQ151',
    #early childhood
    'ECD010','ECQ020','ECD070A','ECD070B','ECQ080','ECQ090','WHQ030E','MCQ080E','ECQ150', 
    #hospital access
    'HUQ010','HUQ020','HUQ030','HUQ071','HUD080','HUQ090', 
    #health status
    'HSD010','HSQ500','HSQ510','HSQ520','HSQ571','HSQ580','HSQ590','HSAQUEX',  
    #income
    'INQ012','INQ030','INQ060','INQ080','INQ090','INQ132','INQ140','INQ150',
    'IND235','INDFMMPI','INDFMMPC', 
    #housing
    'SEQN','HOD050','HOQ065', 
    #occupation
    'OCD150','OCQ180','OCQ210','OCQ260','OCD270','OCQ380','OCD390G','OCD395', 
    #mental health
    'DPQ010','DPQ020','DPQ030','DPQ040','DPQ050','DPQ060','DPQ070','DPQ080','DPQ090','DPQ100', 
    #demographic
    'SDDSRVYR','RIDSTATR','RIAGENDR','RIDAGEYR','RIDAGEMN','RIDRETH1','RIDRETH3',
    'RIDEXMON','RIDEXAGM','DMQMILIZ','DMQADFC','DMDBORN4','DMDCITZN','DMDYRSUS',
    'DMDEDUC3','DMDEDUC2','DMDMARTL','RIDEXPRG','SIALANG','SIAPROXY','SIAINTRP',
    'FIALANG','FIAPROXY','FIAINTRP','MIALANG','MIAPROXY','MIAINTRP','AIALANGA',
    'DMDHHSIZ','DMDFMSIZ','DMDHHSZA','DMDHHSZB','DMDHHSZE','DMDHRGND','DMDHRAGE',
    'DMDHRBR4','DMDHREDU','DMDHRMAR','DMDHSEDU','WTINT2YR','WTMEC2YR','SDMVPSU',
    'SDMVSTRA','INDHHIN2','INDFMIN2','INDFMPIR', 
    #diet nutrition
    'DBQ010','DBD030','DBD041','DBD050','DBD055','DBD061','DBQ073A','DBQ073B','DBQ073C',
    'DBQ073D','DBQ073E','DBQ073U','DBQ700','DBQ197','DBQ223A','DBQ223B','DBQ223C',
    'DBQ223D','DBQ223E','DBQ223U','DBQ229','DBQ235A','DBQ235B','DBQ235C','DBQ301',
    'DBQ330','DBQ360','DBQ370','DBD381','DBQ390','DBQ400','DBD411','DBQ421','DBQ424',
    'DBD895','DBD900','DBD905','DBD910', 
    #drug use
    'DUQ200','DUQ210','DUQ211','DUQ213','DUQ215Q','DUQ215U','DUQ217','DUQ219','DUQ220Q',
    'DUQ220U','DUQ230','DUQ240','DUQ250','DUQ260','DUQ270Q','DUQ270U','DUQ272','DUQ280',
    'DUQ290','DUQ300','DUQ310Q','DUQ310U','DUQ320','DUQ330','DUQ340','DUQ350Q','DUQ350U',
    'DUQ352','DUQ360','DUQ370','DUQ380A','DUQ380B','DUQ380C','DUQ380D','DUQ380E',
    'DUQ390','DUQ400Q','DUQ400U','DUQ410','DUQ420', 
    
]

label = 'DIQ010'

In [5]:
final_features = []

for feature in features:
    if feature not in final_features and feature != 'SEQN':
        final_features.append(feature)

In [6]:
all_data_df['label'] = all_data_df[label].apply(lambda x: x == 1)
all_data_df[[label, 'label']].sample(10)

,DIQ010,label
19811,2.0,False
1173,2.0,False
25452,2.0,False
9741,2.0,False
19777,NaN,False
12905,2.0,False
15978,2.0,False
29144,NaN,False
18289,2.0,False
8693,2.0,False


In [7]:
X = all_data_df[final_features]
y = all_data_df['label']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [9]:
X_train.shape

(23921, 289)

In [10]:
X_test.shape

(2991, 289)

In [11]:
X_val.shape

(2990, 289)

In [19]:
example_pipe = Pipeline([
    ('imp', SimpleImputer(missing_values=np.nan, strategy='mean')), 
    ('scaler', StandardScaler()), 
    ('svc', SVC())
])

In [20]:
example_pipe.fit(X_train, y_train)

Pipeline(steps=[('imp', SimpleImputer()), ('scaler', StandardScaler()),
                ('svc', SVC())])

In [21]:
predictions_train = example_pipe.predict(X_train)
predictions_val = example_pipe.predict(X_val)

In [24]:
roc_auc_score(y_train, predictions_train)

0.9997245179063361

In [25]:
roc_auc_score(y_val, predictions_val)

0.9350393700787402

In [30]:
logreg_pipe = Pipeline([
    ('imp', SimpleImputer(missing_values=np.nan, strategy='mean')), 
    ('scaler', StandardScaler()), 
    ('logreg', LogisticRegression())
])

In [31]:
logreg_pipe.fit(X_train, y_train)

/Volumes/Active/GitHub/ai4all_nhanes/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('imp', SimpleImputer()), ('scaler', StandardScaler()),
                ('logreg', LogisticRegression())])

In [32]:
predictions_train = logreg_pipe.predict(X_train)
predictions_val = logreg_pipe.predict(X_val)

In [33]:
roc_auc_score(y_train, predictions_train)

0.8300826670594648

In [34]:
roc_auc_score(y_val, predictions_val)

0.811853904775061